In [29]:
import os
from pathlib import Path
from PIL import Image
import cv2
import numpy as np
import json

In [2]:
# DeepLab code:
# taken from https://gluon-cv.mxnet.io/build/examples_segmentation/demo_deeplab.html
# dataset description https://groups.csail.mit.edu/vision/datasets/ADE20K/, https://github.com/dmlc/gluon-cv/blob/master/gluoncv/data/ade20k/segmentation.py
# deeplab code https://github.com/dmlc/gluon-cv/blob/master/gluoncv/model_zoo/deeplabv3.py

import mxnet as mx
from mxnet import image
from mxnet.gluon.data.vision import transforms
import gluoncv
from gluoncv.data.transforms.presets.segmentation import test_transform

# using cpu
ctx = mx.cpu(0)

In [15]:
# OCR code
import pytesseract
from pytesseract import Output


# If you don't have tesseract executable in your PATH, include the following:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' # Path('C:/Program Files\ Tesseract-OCR\ tesseract').as_posix()


In [4]:
# Scene recognition imports
from Keras_VGG16_places365.vgg16_places_365 import VGG16_Places365
from cv2 import resize


Using TensorFlow backend.


In [5]:
# Pre-processing functions for tesseract (OCR)
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]


def prepare_dataset(path_image_folder, model_type):
    list_image = []
    # Go through the directory
    list_image_names = []
    pathlist = Path(path_image_folder).glob('**/*.*')
    for path in pathlist:
        # because path is object not string
        path_in_str = str(path)
        list_image_names.append(path.stem)
        # print(path_in_str)
        
        # check for which model the data will be used and pre-process accordingly.
        if model_type == 'deeplab':
            img = image.imread(path_in_str)
            img = test_transform(img, ctx)
        
        elif model_type == 'OCR':
            img = cv2.imread(path_in_str)
            img = get_grayscale(img)
            img = thresholding(img)
        
        elif model_type == 'vgg_places365':
            img = Image.open(path_in_str)
            img = np.array(img, dtype=np.uint8)
            img = resize(img, (224, 224))
            img = np.expand_dims(img, 0)

        list_image.append(img)

    return list_image, list_image_names

In [6]:
image_list_deeplab, list_image_names_deeplab = prepare_dataset('../small_test2017/', 'deeplab')

In [7]:
image_list_OCR, list_image_names_OCR = prepare_dataset('../small_test2017/', 'OCR')

In [8]:
image_list_scene, list_image_names_scene = prepare_dataset('../small_test2017/', 'vgg_places365')

In [ ]:
# TODO: check that the images are ordred similarly.

In [9]:
def load_model(model_type):
    if model_type == 'deeplab':
        model = gluoncv.model_zoo.get_model('deeplab_resnet101_ade', pretrained=True)
    return model

In [10]:
# ADE dataset classes # numbered from 0!
CLASSES = ("wall", "building, edifice", "sky", "floor, flooring", "tree",
               "ceiling", "road, route", "bed", "windowpane, window", "grass",
               "cabinet", "sidewalk, pavement",
               "person, individual, someone, somebody, mortal, soul",
               "earth, ground", "door, double door", "table", "mountain, mount",
               "plant, flora, plant life", "curtain, drape, drapery, mantle, pall",
               "chair", "car, auto, automobile, machine, motorcar",
               "water", "painting, picture", "sofa, couch, lounge", "shelf",
               "house", "sea", "mirror", "rug, carpet, carpeting", "field", "armchair",
               "seat", "fence, fencing", "desk", "rock, stone", "wardrobe, closet, press",
               "lamp", "bathtub, bathing tub, bath, tub", "railing, rail", "cushion",
               "base, pedestal, stand", "box", "column, pillar", "signboard, sign",
               "chest of drawers, chest, bureau, dresser", "counter", "sand", "sink",
               "skyscraper", "fireplace, hearth, open fireplace", "refrigerator, icebox",
               "grandstand, covered stand", "path", "stairs, steps", "runway",
               "case, display case, showcase, vitrine",
               "pool table, billiard table, snooker table", "pillow",
               "screen door, screen", "stairway, staircase", "river", "bridge, span",
               "bookcase", "blind, screen", "coffee table, cocktail table",
               "toilet, can, commode, crapper, pot, potty, stool, throne",
               "flower", "book", "hill", "bench", "countertop",
               "stove, kitchen stove, range, kitchen range, cooking stove",
               "palm, palm tree", "kitchen island",
               "computer, computing machine, computing device, data processor, "
               "electronic computer, information processing system",
               "swivel chair", "boat", "bar", "arcade machine",
               "hovel, hut, hutch, shack, shanty",
               "bus, autobus, coach, charabanc, double-decker, jitney, motorbus, "
               "motorcoach, omnibus, passenger vehicle",
               "towel", "light, light source", "truck, motortruck", "tower",
               "chandelier, pendant, pendent", "awning, sunshade, sunblind",
               "streetlight, street lamp", "booth, cubicle, stall, kiosk",
               "television receiver, television, television set, tv, tv set, idiot "
               "box, boob tube, telly, goggle box",
               "airplane, aeroplane, plane", "dirt track",
               "apparel, wearing apparel, dress, clothes",
               "pole", "land, ground, soil",
               "bannister, banister, balustrade, balusters, handrail",
               "escalator, moving staircase, moving stairway",
               "ottoman, pouf, pouffe, puff, hassock",
               "bottle", "buffet, counter, sideboard",
               "poster, posting, placard, notice, bill, card",
               "stage", "van", "ship", "fountain",
               "conveyer belt, conveyor belt, conveyer, conveyor, transporter",
               "canopy", "washer, automatic washer, washing machine",
               "plaything, toy", "swimming pool, swimming bath, natatorium",
               "stool", "barrel, cask", "basket, handbasket", "waterfall, falls",
               "tent, collapsible shelter", "bag", "minibike, motorbike", "cradle",
               "oven", "ball", "food, solid food", "step, stair", "tank, storage tank",
               "trade name, brand name, brand, marque", "microwave, microwave oven",
               "pot, flowerpot", "animal, animate being, beast, brute, creature, fauna",
               "bicycle, bike, wheel, cycle", "lake",
               "dishwasher, dish washer, dishwashing machine",
               "screen, silver screen, projection screen",
               "blanket, cover", "sculpture", "hood, exhaust hood", "sconce", "vase",
               "traffic light, traffic signal, stoplight", "tray",
               "ashcan, trash can, garbage can, wastebin, ash bin, ash-bin, ashbin, "
               "dustbin, trash barrel, trash bin",
               "fan", "pier, wharf, wharfage, dock", "crt screen",
               "plate", "monitor, monitoring device", "bulletin board, notice board",
               "shower", "radiator", "glass, drinking glass", "clock", "flag")

In [11]:
def get_predictions(input_data, model_type, loaded_model=''):
    
    if model_type == 'vgg_places365':
        file_name = Path('Keras_VGG16_places365/categories_places365.txt')
        if not os.access(file_name, os.W_OK):
            synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
            os.system('wget ' + synset_url)
        classes = list()
        with open(file_name) as class_file:
            for line in class_file:
                classes.append(line.strip().split(' ')[0][3:])
        classes = tuple(classes)
    
    output = []
    for img in input_data:
        if model_type == 'deeplab':
            pred = loaded_model.predict(img)
            # Check what the outputs of predict are: is it probability? does it depend on the class or is it class-agnostic?
            idx_labels = mx.nd.squeeze(mx.nd.argmax(pred, 1)).asnumpy()
            # Decide later whether we make it into actual labels.
            output.append((pred, idx_labels)) # this gives both the prediction "confidence" and the final label (in idx).
        elif model_type == 'OCR': 
            pred = pytesseract.image_to_data(img, output_type=Output.DICT)
            prediction_list = []
            # Get the boundix boxes around the words
            n_boxes = len(pred['text'])
            for i in range(n_boxes):
                # if int(pred['conf'][i]) > 60: to filter per confidence ! see later!
                (x, y, w, h) = (pred['left'][i], pred['top'][i], pred['width'][i], pred['height'][i])
                prediction_list.append((pred['text'][i], (x, y, w, h)))
            output.append(prediction_list)
        elif model_type == 'vgg_places365':
            model = VGG16_Places365(weights='places')
            #predictions_to_return = 5
            preds = model.predict(img)[0]
            top_preds = np.argsort(preds)[::-1]#[0:predictions_to_return]
            top_preds_score = [preds[i] for i in top_preds]
            prediction_list = []
            for i in range(0, len(top_preds)):
                prediction_list.append((classes[top_preds[i]], top_preds_score[i]))
            output.append(prediction_list)

    return output
        
    # Add post processing to reshape the image / bounding boxes /  to original size
    # It seems there's no need for that becaause only the scene recognition model needs resizing.

In [12]:
model_deeplab = load_model('deeplab')
output_pred_deeplab = get_predictions(image_list_deeplab, 'deeplab', model_deeplab)

In [16]:
output_pred_ocr = get_predictions(image_list_OCR, 'OCR')

In [17]:
output_pred_scene = get_predictions(image_list_scene, 'vgg_places365')

In [ ]:
# Evaluation

def evaluate_privacy(ground_truth, predictions, overlap_treshold):
    ### Read ground truth 
    # Go through each private element in GT
    # Go through each private element in predictions
    # Compute overlap
    # Get the max and compare with threshold
    for private_elem in ground_truth
    return list_perf_per_private_element

def evaluate_instance(ground_truth, predictions, segmentation_size):
    
    return list_perf_per_private_element

def evaluate(ground_truth, predictions, evaluation_type, parameter_interval):
    if evaluation_type == 'privacy_type':
        for param_eval in parameter_interval:
            type_private_elements = []
            for idx_im in range(len(predictions)):
                # Get the ground truth 
                # Name of the image
                im_name = list_image_names_deeplab[idx_im]
                result = evaluate_privacy(ground_truth[im_name], predictions[idx_im])
                result.append(type_private_elements)
            print("TODO")
            # And aggreagte
    elif evaluation_type == 'instance_type':
        for param_eval in parameter_interval:
            print("TODO")
        # And aggregate

In [30]:
# Read ground truth file and get the actual annotations
with open(Path('../test2017.json'), 'r') as f:
    ground_truth = json.load(f)
ground_truth = ground_truth['annotations']